## Autoencoder on  2d Müller-Brown potential



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.cm as cm

### 2d Müller-Brown potential

We define the potential $V$ as a class and a function for sampling the trajectory of the Brownian dynamics

$$
dX_t = - \nabla V(X_t) dt + \sqrt{2\beta^{-1}} dW_t
$$

In [ ]:
# Mueller-Brown potential $V(x)$ in 2d
class MuellerPotential:
    def __init__(self, *argv):
        
        # Parameters in the definition of V
        self.a = [-1, -1, -6.5, 0.7]
        self.b = [0, 0, 11, 0.6]
        self.c = [-10, -10, -6.5, 0.7]
        self.A = [-200, -100, -170, 15]
        self.xc = [1, 0, -0.5, -1]
        self.yc = [0, 0.5, 1.5, 1]

        self.x_domain = [-1.8, 1.2]
        self.y_domain = [-0.5, 2.2]
        self.v_min_max = [-130, 20]
        self.contour_levels = [-130, -100, -80, -60, -40, -20, 0.0]
        self.density_max = 0.35
        
    # the potential    
    def V(self, x):
        s = 0
        for i in range(4):
            dx = x[0] - self.xc[i]
            dy = x[1] - self.yc[i]
            s += self.A[i] * np.exp(self.a[i] * dx**2 + self.b[i] * dx * dy + self.c[i] * dy**2)
        return s
    
    # gradient of the potential    
    def gradV(self, x):
        s = 0
        dVx = 0
        dVy = 0
        for i in range(4):
            dx = x[0] - self.xc[i]
            dy = x[1] - self.yc[i]            
            dVx += self.A[i] * (2 * self.a[i] * dx + self.b[i] * dy) * np.exp(self.a[i] * dx**2 + self.b[i] * dx * dy + self.c[i] * dy**2)
            dVy += self.A[i] * (self.b[i] * dx + 2 * self.c[i] * dy) * np.exp(self.a[i] * dx**2 + self.b[i] * dx * dy + self.c[i] * dy**2)
        return np.array((dVx, dVy))

# sample the SDE using Euler-Maruyama scheme

def sample(pot, beta=1.0, delta_t = 0.001, N=10000, seed=42):
    rng = np.random.default_rng(seed=seed)
     
    X = [-0.6, 1.2]
    dim = 2 
    traj = []
    save = 100
    tlist = []
    for i in tqdm(range(N)):
        b = rng.normal(size=(dim,))
        X = X - pot.gradV(X) * delta_t + np.sqrt(2 * delta_t/beta) * b
        if i % save==0:
            traj.append(X)
            tlist.append(i * delta_t)

    return np.array(tlist), np.array(traj)

### define an object of the potential class

In [ ]:
pot = MuellerPotential()  

### visualize the potential

The potenial has two deep local minimum points, which are separated by a shallow local minimum point.

In [ ]:
nx = 100
ny = 150

dx = (pot.x_domain[1] - pot.x_domain[0]) / nx
dy = (pot.y_domain[1] - pot.y_domain[0]) / ny

gridx = np.linspace(pot.x_domain[0], pot.x_domain[1], nx)
gridy = np.linspace(pot.y_domain[0], pot.y_domain[1], ny)
x_plot = np.outer(gridx, np.ones(ny)) 
y_plot = np.outer(gridy, np.ones(nx)).T 

# get grid points
x2d = np.concatenate((x_plot.reshape(nx * ny, 1), y_plot.reshape(nx * ny, 1)), axis=1)

# compute the potential $V$ at grid points
pot_on_grid = np.array([pot.V(x) for x in x2d]).reshape(nx, ny)

print ( "min and max values of potential: (%.4f, %.4f)" % (pot_on_grid.min(), pot_on_grid.max()) )

fig = plt.figure(figsize=(9,5))
ax0 = fig.add_subplot(1, 1, 1)

# visualize the potential and its contour lines
im = ax0.pcolormesh(x_plot, y_plot, pot_on_grid, cmap='coolwarm', vmin=pot.v_min_max[0], vmax=pot.v_min_max[1])
contours = ax0.contour(x_plot, y_plot, pot_on_grid,  pot.contour_levels)
ax0.clabel(contours, inline=True, fontsize=13,colors='black')

ax0.set_aspect('equal')
ax0.set_xlabel(r'$x_1$',fontsize=20)
ax0.set_ylabel(r'$x_2$',fontsize=20, rotation=0)
ax0.tick_params(axis='both', labelsize=20)

ax0.set_xticks([-1.5, -1.0, -0.5, 0, 0.5, 1.0])
ax0.set_yticks([-0.5, 0, 0.5, 1.0, 1.5, 2.0])
ax0.set_xlim([pot.x_domain[0], pot.x_domain[1]])
ax0.set_ylim([pot.y_domain[0], pot.y_domain[1]])

ax0.set_title("Meuller-Brown potential",fontsize=20)
cbar = fig.colorbar(im, ax=ax0, shrink=1.0)
cbar.ax.tick_params(labelsize=15)

plt.show()

### generate a long trajectory of the Brownian dynamics

$$
dX_t = - \nabla V(X_t) dt + \sqrt{2\beta^{-1}} dW_t
$$

We will use the trajectory data to train the autoencoder.

In [ ]:
tlist, trajectory = sample(pot, beta=0.1, delta_t=0.0002, N=5000000)

print ('shape of the trajectory data:', trajectory.shape)

### plot the trajectory data

In [ ]:
fig = plt.figure(figsize=(12,3))

ax1 = fig.add_subplot(1, 3, 1)
ax2 = fig.add_subplot(1, 3, 2)
ax3 = fig.add_subplot(1, 3, 3)

nx = ny = 200

dx = (pot.x_domain[1] - pot.x_domain[0]) / nx
dy = (pot.y_domain[1] - pot.y_domain[0]) / ny
gridx = np.linspace(pot.x_domain[0], pot.x_domain[1], nx)
gridy = np.linspace(pot.y_domain[0], pot.y_domain[1], ny)
x_plot = np.outer(gridx, np.ones(ny)) 
y_plot = np.outer(gridy, np.ones(nx)).T 

# get grid points
x2d = np.concatenate((x_plot.reshape(nx * ny, 1), y_plot.reshape(nx * ny, 1)), axis=1)

# evaluate potential on grid points
pot_on_grid = np.array([pot.V(x) for x in x2d]).reshape(nx, ny)
# plot contour lines of the potential
contours = ax1.contour(x_plot, y_plot, pot_on_grid, levels=pot.contour_levels, cmap='coolwarm')

# scatter plot of the trajectory data
ax1.scatter(trajectory[:,0], trajectory[:,1], alpha=0.5, c='k', s=4)

ax1.set_xlim([pot.x_domain[0], pot.x_domain[1]])
ax1.set_ylim([pot.y_domain[0], pot.y_domain[1]])
ax1.set_title('trajectory')

# plot time-series of the x component
ax2.plot(tlist, trajectory[:,0])
ax2.set_ylim([pot.x_domain[0], pot.x_domain[1]])
ax2.set_title('x coodinate along trajectory')

# plot time-series of the y component
ax3.plot(tlist, trajectory[:,1])
ax3.set_ylim([pot.y_domain[0], pot.y_domain[1]])
ax3.set_title('y coodinate along trajectory')

plt.show()

### define the autoencoder as a neural network

The network implements an encoder $\xi: \mathbb{R}^2\rightarrow \mathbb{R}$ and a decoder $\varphi:  \mathbb{R}\rightarrow \mathbb{R}^2$. 

The autoencoder is defined as $f(x)=\varphi(\xi(x))$.

In [ ]:
# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # encoder
        self.encoder = nn.Sequential(
            nn.Linear(2, 128),
            nn.Tanh(),
            nn.Linear(128, 1),
        )
        
        # decoder
        self.decoder = nn.Sequential(
            nn.Linear(1, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )
    # the autoencoder is the composition of encoder and decoder 
    def forward(self, x):
        z = self.encoder(x)
        x = self.decoder(z)
        return x

### define the model 

In [ ]:
# set the seed, so that results are reproducible.
g = torch.manual_seed(45)

model = Autoencoder()

print (model)

### visualize the untrained autoencoder

In [ ]:
nx = ny = 200

dx = (pot.x_domain[1] - pot.x_domain[0]) / nx
dy = (pot.y_domain[1] - pot.y_domain[0]) / ny
gridx = np.linspace(pot.x_domain[0], pot.x_domain[1], nx)
gridy = np.linspace(pot.y_domain[0], pot.y_domain[1], ny)
x_plot = np.outer(gridx, np.ones(ny)) 
y_plot = np.outer(gridy, np.ones(nx)).T 

x2d = np.concatenate((x_plot.reshape(nx * ny, 1), y_plot.reshape(nx * ny, 1)), axis=1)

pot_on_grid = np.array([pot.V(x) for x in x2d]).reshape(nx, ny)

with torch.no_grad():
    grid_tensor = torch.tensor(x2d, dtype=torch.float32)
    # evaluate encoder on grid points
    latent_values = model.encoder(grid_tensor).numpy().reshape(nx, ny)

# show trajectory data    
plt.scatter(trajectory[:,0], trajectory[:,1], alpha=0.5, c='k', s=4)

plt.xlim([pot.x_domain[0], pot.x_domain[1]])
plt.ylim([pot.y_domain[0], pot.y_domain[1]])
plt.title("Encoder and decoder",fontsize=15)

# plot the contour lines of encoder
contours = plt.contour(x_plot, y_plot, latent_values, levels=20, cmap="viridis")
plt.clabel(contours, inline=True, fontsize=13,colors='black')

# compute the decoded states for different z in latent space
z = torch.linspace(-1.6, 4.4, 50).reshape(-1, 1)
decoded = model.decoder(z).detach().numpy()

# plot the decoded states
plt.scatter(decoded[:,0], decoded[:,1], c='b')

plt.show()

### train the autoencoder with reconstruction loss

In [ ]:
n_epochs = 100
batch_size = 128

train_losses = []

optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loader = DataLoader(torch.tensor(trajectory, dtype=torch.float32), batch_size=batch_size, shuffle=True)

for epoch in tqdm(range(n_epochs)):  # Max epochs
    model.train()
    epoch_train_loss = 0
    
    # Train on minibatches
    for batch_data in train_loader:
        optimizer.zero_grad()
        
        reconstructions = model(batch_data)
        # reconstruction loss
        loss = torch.mean(torch.sum((reconstructions-batch_data)**2, dim=1))
        
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item() * batch_data.size(0)  # Accumulate loss
     
    # Compute average training loss for the epoch
    epoch_train_loss /= len(train_loader.dataset)
    
    # Store losses for plotting
    train_losses.append(epoch_train_loss)

# Plot training and validation loss curves
plt.figure(figsize=(6, 4))
plt.plot(train_losses, label="Train Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()
plt.show()

### visualize the trained encoder and decoder

In [ ]:
fig = plt.figure(figsize=(10,6))

ax0 = fig.add_subplot(1, 2, 1)
ax1 = fig.add_subplot(1, 2, 2)

nx = ny = 200

dx = (pot.x_domain[1] - pot.x_domain[0]) / nx
dy = (pot.y_domain[1] - pot.y_domain[0]) / ny
gridx = np.linspace(pot.x_domain[0], pot.x_domain[1], nx)
gridy = np.linspace(pot.y_domain[0], pot.y_domain[1], ny)
x_plot = np.outer(gridx, np.ones(ny)) 
y_plot = np.outer(gridy, np.ones(nx)).T 

x2d = np.concatenate((x_plot.reshape(nx * ny, 1), y_plot.reshape(nx * ny, 1)), axis=1)

pot_on_grid = np.array([pot.V(x) for x in x2d]).reshape(nx, ny)

# plot the potential and its contour lines
im = ax0.pcolormesh(x_plot, y_plot, pot_on_grid, cmap='coolwarm', vmin=pot.v_min_max[0], vmax=pot.v_min_max[1])
contours = ax0.contour(x_plot, y_plot, pot_on_grid,  pot.contour_levels)
ax0.clabel(contours, inline=True, fontsize=13,colors='black')

ax0.set_aspect('equal')
ax0.set_xlim([pot.x_domain[0], pot.x_domain[1]])
ax0.set_ylim([pot.y_domain[0], pot.y_domain[1]])
ax0.set_title("Meuller-Brown potential",fontsize=15)

with torch.no_grad():
    grid_tensor = torch.tensor(x2d, dtype=torch.float32)
    # evaluate encoder on grid points
    latent_values = model.encoder(grid_tensor).numpy().reshape(nx, ny)

# show trajectory data    
ax1.scatter(trajectory[:,0], trajectory[:,1], alpha=0.5, c='k', s=4)

ax1.set_xlim([pot.x_domain[0], pot.x_domain[1]])
ax1.set_ylim([pot.y_domain[0], pot.y_domain[1]])
ax1.set_aspect('equal')
ax1.set_title("Encoder and decoder",fontsize=15)

# plot the contour lines of encoder
contours = ax1.contour(x_plot, y_plot, latent_values, levels=20, cmap="viridis")
ax1.clabel(contours, inline=True, fontsize=13,colors='black')

# compute the decoded states for different z in latent space
z_min, z_max = np.min(latent_values), np.max(latent_values)
z = torch.linspace(z_min, z_max, 100).reshape(-1, 1)
decoded = model.decoder(z).detach().numpy()

# plot the decoded states
ax1.scatter(decoded[:,0], decoded[:,1], c='b')

plt.show()